> 1. DUPLICATE THIS COLAB DOCUMENT TO START WORKING ON IT: On the top-left corner of this page, go to File > Save a copy to drive.
> 2. SHARE SETTINGS: In the new notebook, set the sharing settings to "Anyone with the link" by clicking "Share" on the top-right corner.

<center>
  <img src=https://www.freevector.com/uploads/vector/preview/31087/07Januari2021-06_generated.jpg width="500" align="center" />
</center>
<br/>

# Week 2: Discover the Airbnb Dataset (and Filter It!)

Hi! 👋👋👋 Are you excited to start the second week's project for Python for Data Science? 

This week's lecture and material on CoRise showed you how to clean a DataFrame and merge one DataFrame into another. As you might have noticed, our dataset this week has way more columns than our dataset from Week 1. This is much more true to real life. It's messy, it's bloated, every time is unique, and above all, it's MUCH MORE interesting!

For this project, we saved an older version of the "clean" **Listings DataFrame** so that you can apply all the steps we performed on the **Calendar DataFrame** directly to this older version and reproduce the methods you learned on slightly different data. Let's get started 💪💪!

## Downloading the Dataset

You'll need to download some prerequisite Python packages in order to run all the code below. Let's install them!

In [1]:
%%capture
!pip install numpy pandas streamlit gdown pyarrow

We will download the datasets from Google Drive just like we did last week, but this time the datasets are in [Pickle](https://pythonnumericalmethods.berkeley.edu/notebooks/chapter11.03-Pickle-Files.html) and [Parquet](https://arrow.apache.org/docs/python/parquet.html) format. 

In [2]:
import os
import shutil

import gdown
import numpy as np
import pandas as pd

# Download files from Google Drive
# Based on data from: http://insideairbnb.com/get-the-data/
file_id_1 = "1m185vTdh-u7_A2ZElBvUD4SCO6oETll2"
file_id_2 = "1w41V1oWHJrBdaNJJQ4oxVBuml5CO7MQX"
downloaded_file_1 = "listings_project.pkl"
downloaded_file_2 = "calendar_project.parquet"
# Download the files from Google Drive
gdown.download(id=file_id_1, output=downloaded_file_1)
gdown.download(id=file_id_2, output=downloaded_file_2)

Downloading...
From: https://drive.google.com/uc?id=1m185vTdh-u7_A2ZElBvUD4SCO6oETll2
To: /content/listings_project.pkl
100%|██████████| 1.42M/1.42M [00:00<00:00, 126MB/s]
Downloading...
From: https://drive.google.com/uc?id=1w41V1oWHJrBdaNJJQ4oxVBuml5CO7MQX
To: /content/calendar_project.parquet
100%|██████████| 1.23M/1.23M [00:00<00:00, 55.6MB/s]


'calendar_project.parquet'

In [3]:
# Show all columns (instead of cascading columns in the middle)
pd.set_option("display.max_columns", None)
# Don't show numbers in scientific notation
pd.set_option("display.float_format", "{:.2f}".format)

## Preprocessing the Dataset
Please load the downloaded files as DataFrames (dfs). The method for loading these datasets is the same as what we did on the CoRise platform.

#### Task 1: Read Pickle and Parquet

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/details-dataset#corise_cl9jxhls800403b6pf9k9nh9e)

Read the Python Pickle and PyArrow Parquet files we've just downloaded as `df_list` and `df_cal`.

In [4]:
df_list = pd.read_pickle(downloaded_file_1)# YOUR CODE HERE
df_cal = pd.read_parquet(downloaded_file_2)# YOUR CODE HERE

Now instead of cleaning the **Calendar DataFrame**, you are going to clean the **Listings DataFrame**. You will use the same steps we used to clean the Calendar data on the CoRise platform this week. Let's first get an overview of the columns that are in this particular DataFrame 🧐.

#### Task 2: Print column names, types, and non-null values 

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/clean-inspect#corise_cl9wmh42v001l3b6pcxidoxp2)

Let's try and get an overview of the **Listings DataFrame**, called `df_list`. This should show us some details about the columns in the DataFrame, like the column names, their data types, and the number of non-null values.

In [5]:
df_list.columns, df_list.dtypes, df_list.isna().sum()

(Index(['id', 'host_acceptance_rate', 'host_is_superhost',
        'host_listings_count', 'host_total_listings_count',
        'neighbourhood_cleansed', 'latitude', 'longitude', 'room_type',
        'accommodates', 'bedrooms', 'beds', 'amenities', 'price',
        'minimum_nights', 'maximum_nights', 'has_availability',
        'availability_30', 'availability_60', 'availability_90',
        'availability_365', 'number_of_reviews', 'number_of_reviews_ltm',
        'number_of_reviews_l30d', 'review_scores_rating', 'instant_bookable',
        'reviews_per_month', 'price_in_euros', 'price_per_person',
        'minimum_price', 'discount_per_5_days_booked',
        'discount_per_10_days_booked', 'discount_per_30_and_more_days_booked',
        'service_cost'],
       dtype='object'), id                                        int64
 host_acceptance_rate                    float64
 host_is_superhost                        object
 host_listings_count                       int64
 host_total_listi

This info printout provides a good overview of which columns we need to investigate further. We saw on CoRise this week that columns with the dtype **object** and sometimes **float** require inspection and cleaning.

Let's start first with the `discount_per_...` columns, where your output should look somthing like this

```
0    5%
1    5%
2    7%
3    6%
4    9%
Name: discount_per_5_days_booked, dtype: object
```

In [6]:
df_list.discount_per_5_days_booked.head(5)

0    5%
1    5%
2    7%
3    6%
4    9%
Name: discount_per_5_days_booked, dtype: object

#### Task 3: Remove, convert, and format

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/clean-inspect#corise_cl9wmqqgx001x3b6p54g1ka2y)

Perform this four-step process to change each of the three `discount_per_...` columns into their proper format:
1. Remove non-numeric characters, like the percent symbol, so you can perform mathematical calculations on the column
1. Change the column into a `float` data type in order to convert the data into a ratio
1. Multiply the whole column by 0.01 so you end up with a probability ratio instead of a percentage
1. Overwrite the old `discount_per_...` column with this new column

Perform these four steps for all thee columns.

***Please note that running this code block more than once might cause an error. This is because you are re-assigning your columns with this code, and if you run the code again, the variable/column you are referring to has already been changed to its preferred state.***


In [7]:
df_list["discount_per_5_days_booked"] = (df_list["discount_per_5_days_booked"]
                                         .str.replace("%", "", regex=True)
                                         .astype('float')
                                         .multiply(0.01)
										 )
df_list["discount_per_10_days_booked"] = (df_list["discount_per_10_days_booked"]
                                         .str.replace("%", "", regex=True)
                                         .astype('float')
                                         .multiply(0.01)
										 )
df_list["discount_per_30_and_more_days_booked"] = (df_list["discount_per_30_and_more_days_booked"]
                                         .str.replace("%", "", regex=True)
                                         .astype('float')
                                         .multiply(0.01)
										 )


Awesome! Let's inspect our results. Your column output should look something like this:

```
0   0.05
1   0.05
2   0.07
3   0.06
4   0.09
Name: discount_per_5_days_booked, dtype: float64
```

In [8]:
df_list.discount_per_5_days_booked.head(5)

0   0.05
1   0.05
2   0.07
3   0.06
4   0.09
Name: discount_per_5_days_booked, dtype: float64

This data looks great for performing calculations!

Next, the columns `host_is_superhost`, `instant_bookable`, and `has_availability` are all boolean columns in the sense that their data represents true and false values, but currently are recognized as objects.

In [9]:
df_list[["host_is_superhost", "instant_bookable", "has_availability"]].head(5)

,host_is_superhost,instant_bookable,has_availability
0,f,t,t
1,t,f,t
2,f,f,t
3,f,f,t
4,t,f,t


This is because the letters in these columns (**t** and **f**) are written as **strings** and not as **boolean** data types. This means we need to replace our string values with the boolean equivalent dtype.

#### Task 4: Booleans!

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/clean-inspect#corise_cl9wmkhbi001r3b6pb2o5kfxd)

Change the columns `host_is_superhost`, `instant_bookable`, and `has_availability` into a boolean data type for better data processing:

1. Replace `f` and `t` with `False` and `True`
1. Set the column as type `bool`
1. Overwrite the old columns with the new values


In [10]:
df_list["host_is_superhost"] = (df_list["host_is_superhost"]
                                .replace({"f": False, "t": True}, inplace=False)
                                .astype('bool')
							    ) # YOUR CODE HERE
df_list["instant_bookable"] = (df_list["instant_bookable"]
                                .replace({"f": False, "t": True}, inplace=False)
                                .astype('bool')
							    ) # YOUR CODE HERE
df_list["has_availability"] = (df_list["has_availability"]
                                .replace({"f": False, "t": True}, inplace=False)
                                .astype('bool')
							    ) # YOUR CODE HERE

Let's now check to confirm we executed these changes correctly. As seen previously,inspecting the different columns should give you an output that looks something like this:


|index|host\_is\_superhost|instant\_bookable|has\_availability|
|---|---|---|---|
|0|false|true|true|
|1|true|false|true|
|2|false|false|true|
|3|false|false|true|
|4|true|false|true|

In [11]:
df_list[["host_is_superhost", "instant_bookable", "has_availability"]].head(5)

,host_is_superhost,instant_bookable,has_availability
0,False,True,True
1,True,False,True
2,False,False,True
3,False,False,True
4,True,False,True


Great, you are making a lot of progress! Let's continue!

#### Task 5: \~\~ Float away ~~

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/clean-inspect#corise_cl9wmqqgx001x3b6p54g1ka2y)

A closer look at the prices in the four columns `price`, `price_per_person`, `minimum_price`, and `service_cost` reveals that they all follow the same pattern:

In [12]:
df_list[["price", "price_per_person", "minimum_price", 'service_cost']].head(5)

,price,price_per_person,minimum_price,service_cost
0,$88.00,$44,$176,$4.99
1,$105.00,$52.5,$315,$4.99
2,$152.00,$38,$304,$4.99
3,$87.00,$43.5,$174,$4.99
4,$160.00,$40,$320,$4.99


All of these four columns have some special characters that you will need to remove before you can change the dtype from object to float.
1. Remove dollar signs and commas
1. Convert to `float`



In [13]:
df_list["price"] = (df_list["price"]
                    .str.replace("$", "", regex=True)
                    .str.replace(",", "", regex=True)
                    .astype('float')
                    )  # YOUR CODE HERE
df_list["price_per_person"] = (df_list["price_per_person"]
                               .str.replace("$", "", regex=True)
                               .str.replace(",", "", regex=True)
                               .astype('float')
                               ) # YOUR CODE HERE
df_list["minimum_price"] = (df_list["minimum_price"]
                            .str.replace("$", "", regex=True)
                            .str.replace(",", "", regex=True)
                            .astype('float')
                            ) # YOUR CODE HERE
df_list["service_cost"] = (df_list["service_cost"]
                           .str.replace("$", "", regex=True)
                           .str.replace(",", "", regex=True)
                           .astype('float')
                           ) # YOUR CODE HERE

Let's inspect the different price columns again and see what it look like, expected output should look like the table below.

|index|price|price\_per\_person|minimum\_price|service\_cost|
|---|---|---|---|---|
|0|88\.0|44\.0|176\.0|4\.99|
|1|105\.0|52\.5|315\.0|4\.99|
|2|152\.0|38\.0|304\.0|4\.99|
|3|87\.0|43\.5|174\.0|4\.99|
|4|160\.0|40\.0|320\.0|4\.99|

In [14]:
df_list[["price", "price_per_person", "minimum_price", 'service_cost']].head(5)

,price,price_per_person,minimum_price,service_cost
0,88.00,44.00,176.00,4.99
1,105.00,52.50,315.00,4.99
2,152.00,38.00,304.00,4.99
3,87.00,43.50,174.00,4.99
4,160.00,40.00,320.00,4.99


#### Task 6: Columns with other names

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/clean-inspect#corise_cl9wmxdm800313b6p8z422k7r)

The following column names need to be changed:
- `price` into `price_in_dollar`
- `neighbourhood_cleansed` into `neighbourhood`  

Please finish the code below.

In [15]:
df_list = df_list.rename(columns={"price": "price_in_dollar", "neighbourhood_cleansed": "neighbourhood"}) # YOUR CODE HERE

#### Task 7: Categories aren't objects

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/clean-inspect#corise_cl9wn60iu003i3b6p65ww9p38)



Taking a closer look at the `neighbourhood` and `room_type` columns reveals that these columns are assigned an **object** dtype. We want them to be a **category** dtype. Please set the correct data type below.

In [16]:
df_list = df_list.astype({'neighbourhood':'category', 'room_type':'category'}) # YOUR CODE HERE

We've made quite a few changes 👌, but we're not done yet! Next we need to delete any columns in the DataFrame that we won't use in our analysis. 

#### Task 8: Delete irrelevant columns

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/clean-inspect#corise_cl9wo15ct005h3b6pbbez3a3u)

It might seem intuitive that more data is always better. But that's not always the case. Often in data science you want *just* the right amount of data — nothing more, nothing less.

We need to delete some columns that are irrelevant to our current use case. Those irrelevant columns are:
* `host_listings_count`
* `host_total_listings_count`
* `availability_60`
* `availability_90`
* `availability_365`
* `number_of_reviews`
* `number_of_reviews_ltm`
* `reviews_per_month`

In [17]:
df_list = df_list.drop(columns=["host_listings_count", "host_total_listings_count",
                                "availability_60", "availability_90",
                                "availability_365", "number_of_reviews",
                                "number_of_reviews_ltm", "reviews_per_month"
                                ]) # YOUR CODE HERE

## -- Your Progress --

Let's now have a look at which data types we still need to change and which columns have some null values.

In [18]:
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6165 entries, 0 to 6172
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    6165 non-null   int64   
 1   host_acceptance_rate                  5365 non-null   float64 
 2   host_is_superhost                     6165 non-null   bool    
 3   neighbourhood                         6165 non-null   category
 4   latitude                              6165 non-null   float64 
 5   longitude                             6165 non-null   float64 
 6   room_type                             6165 non-null   category
 7   accommodates                          6165 non-null   int64   
 8   bedrooms                              5859 non-null   float64 
 9   beds                                  6082 non-null   float64 
 10  amenities                             6165 non-null   int64   
 11  pric

We can see from the output above that the columns `host_acceptance_rate`, `review_scores_rating`, `bedrooms`, `beds`, and `price_in_euros` still require some processing, as they contain missing values, and/or have dtypes like object or float when they need an integer data type.

To summarize, your cleanup so far has reduced memory usage by almost half, which admittedly given the current amount of memory we are using does not really matter that much. But more importantly, our DataFrame is also more readable now, and we can continue to process it in the next sections.

#### Task 9: No unique values

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/clean-inspect#corise_cl9wo1sr2005i3b6phwt2cvvg)

Let's inspect the `price_in_euros` column first, because this column seems to contain only null values, which inherently do not add any meaning to our dataset. Remember our `unique` approach?

In [19]:
df_list["price_in_euros"].nunique() # YOUR CODE HERE

0

The approach should reveal that this column contains no unique values and is thus empty. Please `drop` this column.

In [20]:
df_list = df_list.drop(columns = "price_in_euros") # YOUR CODE HERE

#### Task 10: Dropping rows

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/clean-inspect#corise_cl9wntsuo004t3b6pff12ek7d)

DataFrame [`info()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html) revealed that some listings have no reviews and an unknown host acceptance rate. Most Airbnb users exclude such listings from their search results. To mimic this filtering approach, please filter out any rows that do not have a `review_scores_rating` and without a `host_acceptance_rate`.

A useful method for this approach is the [`dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) function.

In [21]:
df_list = df_list.dropna(subset=["review_scores_rating", "host_acceptance_rate"]) # YOUR CODE HERE

#### Task 11: Reason through your missing data

After setting the right data types, you are often left with making some hard decisions and assumptions about any *partially* incomplete data in your working dataset. In this case, some `beds` and `bedrooms` have no properly assigned values. You can check this by running `df_list.info(verbose=True, show_counts=True)`, which will show that `beds` and `bedrooms` have some missing values.


Let's try and make some simple assumptions based on the `room_type` assigned to the listing. First, inspect which room types are found in the dataset. 

In [22]:
df_list["room_type"].unique()

['Private room', 'Entire home/apt', 'Hotel room', 'Shared room']
Categories (4, object): ['Entire home/apt', 'Hotel room', 'Private room', 'Shared room']

There are four room types. Let's make the assumption that the columns `bedrooms` and `beds` are potentially influenced by `room_type`.

Therefore, we can make the following rules:
- If you have a **"Private room"** or **"Shared room"** as `room_type`, then we believe the listing only has one bedroom. 
- If the listing has **"Hotel room"** or **"Entire home/apt"** as `room_type`, then we can divide the number of guests the listing accomodates by 2 and round up. 
- If any of these numbers are missing, then we can leave it empty.

Translate these requirements into a Python function, and you get:

```python
def fill_empty_bedrooms(accommodates: int, bedrooms: int, room_type: str) -> int:
    if (room_type == "Private room") or (room_type == "Shared room"):
        return 1
    elif (room_type == "Hotel room") or (room_type == "Entire home/apt"):
        return np.ceil(accommodates / 2)
    else:
        return bedrooms
```

Let's time this function and see its performance 💪💪! Please run both cells.

In [23]:
def fill_empty_bedrooms(accommodates: int, bedrooms: int, room_type: str) -> int:
    if (room_type == "Private room") or (room_type == "Shared room"):
        return 1
    elif (room_type == "Hotel room") or (room_type == "Entire home/apt"):
        return np.ceil(accommodates / 2)
    else:
        return bedrooms

In [24]:
%%timeit -r 4 -n 100

temp_df = df_list.copy()  # Deep copy of the df, not a "view"
temp_df["rooms"] = df_list[["accommodates", "bedrooms", "room_type"]].apply(
    lambda x: fill_empty_bedrooms(x["accommodates"], x["bedrooms"], x["room_type"]),
    axis=1,
)

109 ms ± 29.5 ms per loop (mean ± std. dev. of 4 runs, 100 loops each)


Just like in Week 1, we use `timeit` to measure the performance of our function. In the case of Pandas, we are using [`apply()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) to semi-vectorize our function, but secretly this function just implements something that mimics a for loop. Using [a lambda](https://www.geeksforgeeks.org/applying-lambda-functions-to-pandas-dataframe/) together with [`apply()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) allows us to access multiple columns to generate an outcome.


This approach is often good enough, but not always, especially if you are dealing with large datasets. Below we will run the [`apply()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) function for output. We've also provided some alternative functions, though they are often found to be slower. (However, it can be a bit of a grey area, [so it really depends](https://stackoverflow.com/questions/24870953/does-pandas-iterrows-have-performance-issues/24871316#24871316).)

In [25]:
df_list["bedrooms"].head()
#%%
df_list["bedrooms"] = df_list[["accommodates", "bedrooms", "room_type"]].apply(
    lambda x: fill_empty_bedrooms(x["accommodates"], x["bedrooms"], x["room_type"]),
    axis=1,)# YOUR CODE HERE

| Related functions ([In *general* order of preference](https://stackoverflow.com/questions/24870953/does-pandas-iterrows-have-performance-issues/24871316#24871316))|
| ---- |
| [apply()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html): Apply a function along one or multiple columns |
| [pipe()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pipe.html): Chain multiple transformations/functions after each other |
| [applymap()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.applymap.html): Use strictly as a transformation of current value to a new value |
| [itertuples()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.itertuples.html): Iterate over DataFrame rows as named tuples |
| [iteritems()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iteritems.html): Iterate  over DataFrame columns |
| [iterrows()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iterrows.html): Iterate over DataFrame rows |

If you feel like you want to take the function's performance to the next level, we have a **bonus** (**NOT** required) question 😱!

---

#### (Extra Credit) Task 12: Vectorize!

Can you vectorize the function by using the method described under "Pandas Vectorization", with inspiration from [this link](https://towardsdatascience.com/do-you-use-apply-in-pandas-there-is-a-600x-faster-way-d2497facfa66)? On our hardware, applying vectorization led to results that were at least 30-40x faster. (This might differ a bit on your hardware.)

In [26]:
%%timeit -r 4 -n 100

temp_df = df_list.copy()

# Please use as many lines as you think you need to 
# implement this function. We required 5 separate
# statements.
...

201 µs ± 30.8 µs per loop (mean ± std. dev. of 4 runs, 100 loops each)


---

#### Task 13: Clean-up crew

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/clean-inspect#corise_cl9wntsuo004t3b6pff12ek7d)

Thanks to our logic and assumptions, most listings now have a proper amount of defined rooms. However, there are still a few listings without any number of rooms defined. Remove all rows/entries that have an empty `bedrooms`, `beds`.

In [27]:
df_list = df_list.dropna(subset=["bedrooms", "beds"]) # YOUR CODE HERE

Now that we have removed all the empty values, finally we can assign the dtype `int` instead of `float` to these two columns.

Please set the columns `beds` and `bedrooms` as `int`. 

In [28]:
df_list["beds"] = df_list["beds"].astype("int") # YOUR CODE HERE
df_list["bedrooms"] = df_list["bedrooms"].astype("int") # YOUR CODE HERE

<ipython-input-28-ba1ab426afdd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list["beds"] = df_list["beds"].astype("int") # YOUR CODE HERE
<ipython-input-28-ba1ab426afdd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list["bedrooms"] = df_list["bedrooms"].astype("int") # YOUR CODE HERE


---

#### (Extra Credit) Task 14: Speed it up

Lastly, and this is entirely optional, but you can further speed-up data processing by taking the appropriate number of bytes for a given data type, especially when dealing with large datasets (more info [here](https://towardsdatascience.com/reducing-memory-usage-in-pandas-with-smaller-datatypes-b527635830af)). Currently, the data types are set to 64 bits by default, but most of these could be set to lower values. Good luck 💪💪!

---

## Cleaning Is DONE!

<center>
  <img src=https://i.ibb.co/f9wQt8T/2016-celebrate-celebration-city-preview.jpg width="500" align="center" />
</center>
<br/>

✨ Awesome ✨! You cleaned our data and made sure to watch that the intermediate results aligned with what we expected.

In [29]:
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4817 entries, 0 to 6172
Data columns (total 25 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    4817 non-null   int64   
 1   host_acceptance_rate                  4817 non-null   float64 
 2   host_is_superhost                     4817 non-null   bool    
 3   neighbourhood                         4817 non-null   category
 4   latitude                              4817 non-null   float64 
 5   longitude                             4817 non-null   float64 
 6   room_type                             4817 non-null   category
 7   accommodates                          4817 non-null   int64   
 8   bedrooms                              4817 non-null   int64   
 9   beds                                  4817 non-null   int64   
 10  amenities                             4817 non-null   int64   
 11  pric

Using the function [`head()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html) reveals the same. 

In [30]:
df_list.head(3)

,id,host_acceptance_rate,host_is_superhost,neighbourhood,latitude,longitude,room_type,accommodates,bedrooms,beds,amenities,price_in_dollar,minimum_nights,maximum_nights,has_availability,availability_30,number_of_reviews_l30d,review_scores_rating,instant_bookable,price_per_person,minimum_price,discount_per_5_days_booked,discount_per_10_days_booked,discount_per_30_and_more_days_booked,service_cost
0,23726706,0.95,False,IJburg - Zeeburgereiland,52.35,4.98,Private room,2,1,1,6,88.00,2,14,True,0,3,4.99,True,44.00,176.00,0.05,0.11,0.16,4.99
1,35815036,1.00,True,Noord-Oost,52.42,4.96,Entire home/apt,2,1,1,5,105.00,3,100,True,4,6,4.96,False,52.50,315.00,0.05,0.12,0.16,4.99
2,31553121,1.00,False,Noord-West,52.43,4.92,Entire home/apt,4,2,3,3,152.00,2,60,True,0,1,4.74,False,38.00,304.00,0.07,0.11,0.22,4.99


As you might have noticed, the steps you take and the methods you use to clean your data is very dependent on the data that is given. If you'd like more practice, we encourage you to take a look at this [Kaggle tutorial](https://www.kaggle.com/learn/data-cleaning) which shows some other problems that might occur when dealing with data.

## Mix and Match

<center>
  <img src=https://successrice.com/wp-content/uploads/2020/08/Pride-038-1-980x551.jpg.webp width="500" align="center" />
</center>
<br/>

As a next step, you will again merge these two datasets, as was shown in this week's content. However, this time around we will take a slightly different angle with the data.

In [31]:
# The Calendar DataFrame!
df_cal.head(3)

,listing_id,date,available,price_in_dollar,minimum_nights,maximum_nights
0,23726706,2022-06-05,False,90.00,2,1125
1,23726706,2022-06-06,False,90.00,2,1125
2,23726706,2022-06-07,False,90.00,2,1125


---

#### (Extra Credit) Task 15: Minimum stay

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/merging-pros)

You are looking to stay for a minimum of 3 days. Since you are unsure when it will be booked, you'd like to exclude all `listing_ids` that go below that threshold of 3 days no matter what time of year. This is because you think that bookings with a minimum stay of 3 days are more likely to have discount prices.

With these excluded, you would like to see the expected booking price for 5 days: 

- Using the helper code, create a conditional index for entries `listing_id` that are above the 3 day threshold
- Remove them using `.loc[conditional_index]` over the conditional index.
- Calculate the price of booking a listing for 5 days by multiplying the current day multiplied by 5, and assign this to a column called `five_day_dollar_price`

In [32]:
# First start by making a copy, for debugging purposes
calendar_newdf = df_cal.copy()

include_list = (
    calendar_newdf[calendar_newdf["minimum_nights"] < 3]["listing_id"].unique().tolist()
)
# Note: Keeping >=3 cant erase values for 1 or 2 night in the next section

In [33]:
# Get all the listings with a minimum nights of 3+
# Use the include_list
calendar_newdf = calendar_newdf.loc[~calendar_newdf["listing_id"].isin(include_list)] # YOUR CODE HERE
calendar_newdf

,listing_id,date,available,price_in_dollar,minimum_nights,maximum_nights
1095,623123,2022-06-05,False,450.00,4,1125
1096,623123,2022-06-06,False,450.00,4,1125
1097,623123,2022-06-07,False,450.00,4,1125
1098,623123,2022-06-08,False,450.00,4,1125
1099,623123,2022-06-09,False,450.00,4,1125
...,...,...,...,...,...,...
2252045,47709873,2023-05-31,False,213.00,7,120
2252046,47709873,2023-06-01,False,213.00,7,120
2252047,47709873,2023-06-02,False,213.00,7,120
2252048,47709873,2023-06-03,False,213.00,7,120


| Related functions |
| ---- |
| [isin()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html): Filter the DataFrame on provided values |
| [eq()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.eq.html#pandas.DataFrame.eq): Filter the DataFrame for all values equal to the provided input |
| [ne()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ne.html#pandas.DataFrame.ne): Filter the DataFrame for all values not equal to the provided input |

In [34]:
calendar_newdf["five_day_dollar_price"] = (calendar_newdf['price_in_dollar'] * 5) # YOUR CODE HERE

<ipython-input-34-a9c96e20cfce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calendar_newdf["five_day_dollar_price"] = (calendar_newdf['price_in_dollar'] * 5) # YOUR CODE HERE


Now let's transform our newly created DataFrame into a **pivot table**, where we aggregate our rows using the `listing_id` as the index, and the columns `available` and `five_day_dollar_price` as values.

The expected table output should look something like.

|listing\_id|available|five\_day\_dollar\_price|
|---|---|---|
|2818|0\.20821917808219179|346\.90410958904107|
|44391|0\.0|1200\.0|
|49552|0\.4581005586592179|1162\.5|


In [35]:
calendar_summarizeddf = pd.pivot_table(
    data=calendar_newdf,
    index=["listing_id"],
    values=["available", "five_day_dollar_price"],
    aggfunc=np.mean,  # The default aggregation function used
    # for merging multiple related rows of data.
)

calendar_summarizeddf.head(3)

,available,five_day_dollar_price
listing_id,,
2818,0.21,346.90
44391,0.00,1200.00
55709,0.00,818.68


#### (Extra Credit) Task 16: Maximum price and date

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/merging-pros#corise_cl9woiqve006p3b6p0vsdbqge)

Can you make a **pivot table** that states the **maximum `price_in_dollar`** for every Airbnb listing?

The expected pivot table output should look like.

|listing\_id|price\_in\_dollar|
|---|---|
|2818|80\.0|
|44391|240\.0|
|49552|300\.0|

In [40]:
temp_sum_df = pd.pivot_table(
    data=calendar_newdf,
    index=["listing_id"],
    values=["price_in_dollar"],
    aggfunc=np.max,  


)

temp_sum_df.head(3)

,price_in_dollar
listing_id,
2818,80.00
44391,240.00
55709,250.00


---

## Join Us?!

<center>
  <img src=https://upload.wikimedia.org/wikipedia/commons/9/95/Merge_left_%28259959%29_-_The_Noun_Project.svg width="200" align="center" />
</center>
<br/>

We are going to merge the pivot table that includes the `five_day_dollar_price` for each listing with `listings_df`. We have to keep in mind that we want to keep only those rows of Airbnb listing IDs that are present in both datasets.

#### Task 17: Mergin'

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/merging-pros#corise_cl9wp1zf9007m3b6pfiawiagd)

Let's use the [`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) operation as was shown on CoRise, with the pivot table on the right and the **Listings DataFrame** on the left. Make sure to provide which columns you want to join on for our pivot table and the DataFrame.


The expected merged table should look same as

|index|id|host\_acceptance\_rate|host\_is\_superhost|neighbourhood\_cleansed|latitude|longitude|room\_type|accommodates|bedrooms|beds|amenities|price|minimum\_nights|maximum\_nights|has\_availability|availability\_30|number\_of\_reviews\_l30d|review\_scores\_rating|instant\_bookable|price\_per\_person|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|0|35815036|1\.0|true|Noord-Oost|52\.42419|4\.95689|Entire home/apt|2|1|1|5|105\.0|3|100|true|4|6|4\.96|false|52\.5|
|1|19572024|1\.0|false|Watergraafsmeer|52\.30739|4\.90833|Entire home/apt|6|3|6|14|279\.0|3|300|true|6|3|4\.69|false|46\.5|
|2|2973384|0\.38|false|Watergraafsmeer|52\.30989|4\.90528|Entire home/apt|5|3|3|7|185\.0|6|21|true|0|0|4\.83|false|37\.0|

In [41]:
final_df = pd.merge(
        df_list,
        calendar_summarizeddf,
        how="inner",
        right_on = ["listing_id"],
        left_on = ["id"],# YOUR CODE HERE
        )
final_df.head(3)

,id,host_acceptance_rate,host_is_superhost,neighbourhood,latitude,longitude,room_type,accommodates,bedrooms,beds,amenities,price_in_dollar,minimum_nights,maximum_nights,has_availability,availability_30,number_of_reviews_l30d,review_scores_rating,instant_bookable,price_per_person,minimum_price,discount_per_5_days_booked,discount_per_10_days_booked,discount_per_30_and_more_days_booked,service_cost,available,five_day_dollar_price
0,19572024,1.00,False,Watergraafsmeer,52.31,4.91,Entire home/apt,6,3,6,14,279.00,3,300,True,6,3,4.69,False,46.50,837.00,0.09,0.16,0.14,4.99,0.82,1496.55
1,2973384,0.38,False,Watergraafsmeer,52.31,4.91,Entire home/apt,5,3,3,7,185.00,6,21,True,0,0,4.83,False,37.00,1110.00,0.06,0.12,0.18,4.99,0.04,941.30
2,34985473,1.00,True,Noord-Oost,52.43,5.04,Entire home/apt,5,3,4,4,135.00,6,1125,True,7,0,4.43,False,27.00,810.00,0.08,0.13,0.17,4.99,0.16,748.56


---

#### (Extra Credit) Task 18: Groups are great

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/data-explore-stats#corise_cl9wpp3t100n53b6pmv167sm7)

Now, let's perform a [`groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) where we look at the median values of `five_day_dollar_price` and `review_scores_accuracy` with respect to the `room_type`. Do these results match your intuition?


The expected group by table should look same as,

|room\_type|review\_scores\_rating|five\_day\_dollar\_price|
|---|---|---|
|Entire home/apt|4\.88|972\.7397260273973|
|Hotel room|4\.5649999999999995|908\.1575342465753|
|Private room|4\.79|681\.986301369863|
|Shared room|4\.6|565\.027397260274|

In [42]:
... # YOUR CODE HERE

Ellipsis

You might have expected that shared rooms are the cheapest and thus have the lowest rating with respect to median scores. The same can't be said for the most expensive option — a hotel room. Will this influence your future considerations when booking 🤔?

(But before you let this influence your decisions too much, it might be better to assume that this data might be biased in favor of Airbnb and not hotels in general. 🤷)

---

You've walked through all the most important parts of Pandas. It's a really easy-to-use library that shares a lot of syntax with NumPy. It is great for analyzing and cleaning datasets, and as you might have discovered with the previous code, Pandas allows you to really go into the nitty-gritty details of your dataset. These skills are invaluable for a data scientist, and will empower you to utilize data where you work now, or even where you could work in the future!

The next steps involve downloading the files to your local computer so that you can make an app for your portfolio. After, we will provide some suggestions on how you can extend this project, along with some interesting links to investigate.

### Download the Dataset to Your Local Machine

Let's first export our final DataFrame.

In [43]:
final_df.to_csv(
    "WK2_Airbnb_Amsterdam_listings_proj_solution.csv",
    index=True,
)

Google Colab comes with its own Python packages that allow us to quickly download generated files, like so:

In [44]:
from google.colab import files

# Download the file locally
files.download('WK2_Airbnb_Amsterdam_listings_proj_solution.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Make an App for Your Portfolio!

<center>
  <img src=https://griddb-pro.azureedge.net/en/wp-content/uploads/2021/08/streamlit-1160x650.png width="500" align="center" />
</center>
<br/>

**Participants such as yourselves often want to use the weekly CoRise projects for their portfolios. To facilitate that, we've created this section. It might seem like a lot, but it's actually just following instructions and copy-pasting. Reach out on Slack if you get stuck!** 

You will make an app that visualizes the dataset as a DataFrame and as a geographic visualization like so:

<center>
  <img src=https://i.ibb.co/N9JKbd8/Screen-Shot-2022-11-10-at-4-07-17-PM.png width="500" align="center" />
</center>
<br/>

To visualize this, we will again use a library called [Streamlit](https://streamlit.io/). For now you are not expected to know how Streamlit works, but you are expected to be able to copy-paste and follow instructions if you want to share this project as part of your portfolio!

We are going to use [Streamlit Share](https://share.streamlit.io/) to host your projects. It's a website that allows us to host our interactive projects for free online! Again, we don't expect you to understand how to use and/or modify the code shown below. We do expect you to read the instructions and copy-paste our code to the Streamlit Share platform. Feel free to change it any way you like. Some great starting points are [here](https://python.plainenglish.io/how-to-build-web-app-using-streamlit-pandas-numpy-5e134f0cf552), [here](https://docs.streamlit.io/library/get-started/create-an-app), [here](https://streamlit.io/components), and [here](https://streamlit.io/gallery)!

The `app.py` below is based on [this code](https://github.com/tylerjrichards/st-filter-dataframe/blob/main/streamlit_app.py). Feel free to extend it if you like!

In [45]:
%%writefile app.py
import pandas as pd
import streamlit as st
from pandas.api.types import (
    is_categorical_dtype,
    is_datetime64_any_dtype,
    is_numeric_dtype,
    is_object_dtype
)

st.title("Filter your Airbnb Listings dataframe!")

st.write(
    """This app is based on this blog [here](https://blog.streamlit.io/auto-generate-a-dataframe-filtering-ui-in-streamlit-with-filter_dataframe/). 
    Can you think of ways to extend it with visuals?
    """
)


def filter_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds a UI on top of a dataframe to let viewers filter columns
    Args:
        df (pd.DataFrame): Original dataframe
    Returns:
        pd.DataFrame: Filtered dataframe
    """
    modify = st.checkbox("Add filters")

    if not modify:
        return df

    df = df.copy()

    # Try to convert datetimes into a standard format (datetime, no timezone)
    for col in df.columns:
        if is_object_dtype(df[col]):
            try:
                df[col] = pd.to_datetime(df[col])
            except Exception:
                pass

        if is_datetime64_any_dtype(df[col]):
            df[col] = df[col].dt.tz_localize(None)

    modification_container = st.container()

    with modification_container:
        to_filter_columns = st.multiselect("Filter dataframe on", df.columns)
        for column in to_filter_columns:
            left, right = st.columns((1, 20))
            left.write("↳")
            # Treat columns with < 10 unique values as categorical
            if is_categorical_dtype(df[column]) or df[column].nunique() < 10:
                user_cat_input = right.multiselect(
                    f"Values for {column}",
                    df[column].unique(),
                    default=list(df[column].unique()),
                )
                df = df[df[column].isin(user_cat_input)]
            elif is_numeric_dtype(df[column]):
                _min = float(df[column].min())
                _max = float(df[column].max())
                step = (_max - _min) / 100
                user_num_input = right.slider(
                    f"Values for {column}",
                    _min,
                    _max,
                    (_min, _max),
                    step=step,
                )
                df = df[df[column].between(*user_num_input)]
            elif is_datetime64_any_dtype(df[column]):
                user_date_input = right.date_input(
                    f"Values for {column}",
                    value=(
                        df[column].min(),
                        df[column].max(),
                    ),
                )
                if len(user_date_input) == 2:
                    user_date_input = tuple(map(pd.to_datetime, user_date_input))
                    start_date, end_date = user_date_input
                    df = df.loc[df[column].between(start_date, end_date)]
            else:
                user_text_input = right.text_input(
                    f"Substring or regex in {column}",
                )
                if user_text_input:
                    df = df[df[column].str.contains(user_text_input)]

    return df


df = pd.read_csv(
    "WK2_Airbnb_Amsterdam_listings_proj_solution.csv", index_col=0
)
st.dataframe(filter_dataframe(df))

Writing app.py


The **%%writefile [FILE_NAME].[FILE_EXTENSION]** command let's us save the code written in the cells in your Google Colab instance. Having it saved like that enables us to download it as a file, as seen below.

In [46]:
from google.colab import files

# Download the file locally
files.download('app.py')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
%%writefile requirements.txt
pandas
streamlit

Writing requirements.txt


In [48]:
from google.colab import files

# Download the file locally
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Please verify that you've downloaded three files:
- `WK2_Airbnb_Amsterdam_listings_proj_solution.csv`
- `app.py`
- `requirements.txt`

Now let's head over to GitHub and [create an account](https://github.com/signup).

Then, since you are logged in, [go to GitHub.com](https://github.com) and click on the **+** icon at the top-right corner and select **New repository**.

<center>
  <img src=https://i.ibb.co/4gkPBCp/Screen-Shot-2022-11-28-at-1-51-02-PM.png width="300" align="center" />
</center>
<br/>

Here you provide:
- **Repository name**: Up to you
- **License**: Up to you. We recommend **apache-2.0**.

- **Public or private?** Public, otherwise you can't host it on [Streamlit Share](https://share.streamlit.io)!

<center>
  <img src=https://i.ibb.co/0B533dw/Screen-Shot-2022-11-28-at-1-55-14-PM.png width="450" align="center" />
</center>
<br/>

Then upload the three files to this URL below. ***Please modify it before copy-pasting it***.

```https://github.com/[YOUR_ACCOUNT_NAME]/[YOUR_REPOSITORY_NAME]/upload/main```

<center>
  <img src=https://i.ibb.co/jTsrgJw/Screen-Shot-2022-11-28-at-1-58-31-PM.png width="500" align="center" />
</center>
<br/>

Commit directly to the `main` branch, then click **Commit changes**.

Next, you have to create an account on [Streamlit Share](https://share.streamlit.io/signup). 

<center>
  <img src=https://i.ibb.co/znFngJc/Screen-Shot-2022-11-28-at-1-59-47-PM.png width="500" align="center" />
</center>
<br/>

It's recommended that you click **Continue with GitHub**. 

Then, select **New app** **>** **Deploy a new app...** **>** **From existing repo**.

<center>
  <img src=https://i.ibb.co/VQPQzt3/Screen-Shot-2022-11-28-at-2-05-04-PM.png width="500" align="center" />
</center>

Followed by providing your:

```[GITHUB_ACCOUNT_NAME]/[GITHUB_REPOSITORY]```

and your `streamlit_app.py` as `app.py`
<center>
  <img src=https://i.ibb.co/PDSQccD/Screen-Shot-2022-11-28-at-2-10-47-PM.png width="500" align="center" />
</center>

You will have to wait around 1-5 minutes, then an automatic hyperlink is generated for your new website. An example is this app:

```https://[GITHUB_ACCOUNT_NAME]-[GITHUB_REPOSITORY]-[RANDOM_6_LETTER_STRING].streamlit.app/```

***Please modify the link before copy-pasting it.***

---

# 🎉 CONGRATULATIONS!!!

You've made it to the end of the Week 2 assignment! You should be proud. 

If you have any lingering questions, post them on Slack! As you know, we're always here to help.

And if you want any additional challenge questions, check out the bonus extensions below.

---

## Extensions (Optional)

<center>
  <img src=https://raw.githubusercontent.com/aschonfeld/dtale-media/master/images/Title.png width="500" align="center" />
</center>
<br/>

Awesome 🎉🎉🎉 you are finished with all the tasks! How about considering running [D-Tale](https://github.com/man-group/dtale) on your DataFrames, so you can interactively explore your dataset? What about getting a visual representation of the data and thus a quick overview of what's goin' on by using [Lux](https://github.com/lux-org/lux) or [Sweetviz](https://pypi.org/project/sweetviz/)?

If optimizing code gets you excited, then try to see how you can maximize these performance numbers even more. Maybe it's time to try running this code using [cuDF](https://github.com/rapidsai/cudf), [Polars](https://github.com/pola-rs/polars), or [Vaex](https://github.com/vaexio/vaex)?

# Next Up?
This is the last project of our course 😭😭. Please do not be sad! We would like to invite you to read the "What's Next?" post in Week 3, as it contains great suggestions on how you can proceed with your learning journey at CoRise (yes, please!) or somewhere else!